In [2]:
import os
import sys
import requests
from pathlib import Path
import numpy as np
import plotly.express as px
from tqdm import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange, reduce, repeat
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
from fancy_einsum import einsum
import math 

device = t.device("cuda" if t.cuda.is_available() else "cpu")

Create the toy dataset

There are two options for input sequences: noise and signal. The signal contains a 0 preceded by 2 numbers less in [1,4], and has a ground truth answer given by an arbitrary map. 

In [3]:
ans_map = {(1, 1):0, (1, 2):1, (1, 3):2, (1, 4):3, (2, 2):4, (2, 3):5, (2, 4):6, (3,3):7, (3, 4):8, (4, 4):9}
class SkipTrigramDataset(Dataset):

    def __init__(self, size: int, prob:float, prompt_length: int, seed:int = 42):
        self.size = size
        self.prob = prob
        self.prompt_length = prompt_length
        self.data = [self.generate_sample() for _ in range(size)]
        t.manual_seed(seed=seed)

    def generate_sample(self):
        random_tensor = t.randint(low = 5, high = 11, size = (self.prompt_length,))
        random_int = t.randint(low = 0, high = self.prob, size = (1,))
        if random_int.item() == self.prob - 1:
            rand_num_1 = t.randint(1, 5, size = (1,)).item()
            rand_num_2 = t.randint(1, 5, size = (1,)).item()
            random_tensor[0] = rand_num_1
            random_tensor[-2] = rand_num_2
            sample = random_tensor
            sample[-1] = 0
            label = ans_map[tuple(sorted((rand_num_1, rand_num_2)))]
            label = t.tensor([label])
        else:
            sample = random_tensor
            label = t.randint(low = 0, high = 11, size = (1,))

        return sample, label


    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.size

In [4]:
ans_map = {(1, 1):0, (1, 2):1, (1, 3):2, (1, 4):3, (2, 2):4, (2, 3):5, (2, 4):6, (3,3):7, (3, 4):8, (4, 4):9}

example_dataset = SkipTrigramDataset(size = 10_000, prob = 2, prompt_length = 4)
for i in range(10):
    print(f'Sample: {example_dataset[i][0]}, label: {example_dataset[i][1][-1]}')



Sample: tensor([ 7,  9, 10, 10]), label: 3
Sample: tensor([5, 8, 9, 8]), label: 0
Sample: tensor([2, 7, 4, 0]), label: 6
Sample: tensor([3, 6, 4, 0]), label: 8
Sample: tensor([1, 9, 1, 0]), label: 0
Sample: tensor([8, 6, 7, 7]), label: 4
Sample: tensor([2, 7, 2, 0]), label: 4
Sample: tensor([10,  6,  5,  7]), label: 4
Sample: tensor([ 8,  6, 10,  9]), label: 3
Sample: tensor([1, 5, 2, 0]), label: 1


In [5]:
signal = [example[1].item() for example in example_dataset if example[0][-1] == 0]
noise = [example[1].item() for example in example_dataset if example[0][-1] != 0]

print(len(signal), len(noise))

px.histogram(signal)
#px.histogram(noise)

## repeated ints < 5 are half as likely in the signal dataset. This shouldn't matter, but could check to see if those outputs (0, 4, 7, 9) are treated systematically differently by the model


4957 5043


In [6]:
@dataclass
class model_cfg:
    d_model: int = 12
    d_vocab: int = 12
    init_range: float = 0.02
    n_ctx: int = 5
    d_head: int = 3
    n_heads: int = 3
    n_layers: int = 1

model_config = model_cfg()

print(model_config)

model_cfg(d_model=12, d_vocab=12, init_range=0.02, n_ctx=5, d_head=3, n_heads=3, n_layers=1)


In [7]:
class OneHotEmbed(nn.Module):
    def __init__(self, model_cfg):
        super().__init__()
        self.model_cfg = model_cfg
        self.embedding = nn.Embedding(self.model_cfg.d_vocab, self.model_cfg.d_vocab)
        self.embedding.weight = nn.Parameter(t.eye(self.model_cfg.d_vocab))
        self.embedding.weight.requires_grad = False

    def forward(self, input):
        # tokens: [batch, position]
        # output: [batch, position, d_model]
        input = input.to(device)
        embed_out = self.embedding(input)
        return embed_out
    
class OneHotUnembed(nn.Module):
    def __init__(self, model_cfg):
        super().__init__()
        self.model_cfg = model_cfg
        self.unembed = t.eye(self.model_cfg.d_vocab)

    def forward(self, input):
        # tokens: [batch, position, d_model]
        # output: [batch, position]
        input = input.to(device)
        return t.einsum("vm,bpm->bpv", self.unembed.to(device), input)  # [batch, pos, d_vocab]


In [8]:
class Attention(nn.Module):
    def __init__(self, model_cfg):
        super().__init__()
        self.model_cfg = model_cfg

        # set weights, biases as parameters 
        self.W_Q = nn.Parameter(t.empty((model_cfg.n_heads, model_cfg.d_model, model_cfg.d_head)))
        self.W_K = nn.Parameter(t.empty((model_cfg.n_heads, model_cfg.d_model, model_cfg.d_head)))
        self.W_V = nn.Parameter(t.empty((model_cfg.n_heads, model_cfg.d_model, model_cfg.d_head)))
        self.W_O = nn.Parameter(t.empty((model_cfg.n_heads, model_cfg.d_head, model_cfg.d_model)))
        self.b_Q = nn.Parameter(t.zeros((model_cfg.n_heads, model_cfg.d_head)))
        self.b_K = nn.Parameter(t.zeros((model_cfg.n_heads, model_cfg.d_head)))
        self.b_V = nn.Parameter(t.zeros((model_cfg.n_heads, model_cfg.d_head)))
        self.b_O = nn.Parameter(t.zeros((model_cfg.d_model)))

        #initialize randomly
        nn.init.normal_(self.W_Q, std=self.model_cfg.init_range)
        nn.init.normal_(self.W_K, std=self.model_cfg.init_range)
        nn.init.normal_(self.W_V, std=self.model_cfg.init_range)
        nn.init.normal_(self.W_O, std=self.model_cfg.init_range)
        self.register_buffer("IGNORE", t.tensor(-1e5, dtype=t.float32, device=device))

        self.attn_activation_cache = {}

    def forward(self, normalized_resid_pre: t.Tensor):
        # normalized_resid_pre: [batch, position, d_model] (we're not using LN, so just input)
        # output: [batch, position, d_model]
        normalized_resid_pre = normalized_resid_pre.to(device)

        queries = einsum('n_heads d_model d_head, batch pos_q d_model -> batch pos_q n_heads d_head', self.W_Q, normalized_resid_pre) + self.b_Q
        keys = einsum('n_heads d_model d_head, batch pos_k d_model -> batch pos_k n_heads d_head', self.W_K, normalized_resid_pre) + self.b_K
        values = einsum('n_heads d_model d_head, batch pos_k d_model -> batch pos_k n_heads d_head', self.W_V, normalized_resid_pre) + self.b_V

        attn_scores = einsum('batch pos_q n_heads d_head, batch pos_k n_heads d_head -> batch n_heads pos_q pos_k', queries, keys)

        scaled_attn = attn_scores / math.sqrt(self.model_cfg.d_head)
        masked_attn = self.apply_causal_mask(scaled_attn)
        # apply softmax over keys to get attention pattern
        softmax_attn = masked_attn.softmax(dim = -1)

        # get weighted attention values
        z = einsum('batch pos_k n_heads d_head, batch n_heads pos_q pos_k -> batch pos_q n_heads d_head', values, softmax_attn)

        # layer output
        output = einsum('n_heads d_head d_model, batch pos_q n_heads d_head -> batch pos_q d_model', self.W_O, z) + self.b_O

        ## save to the cache 
        self.attn_activation_cache['hook_q'] = queries
        self.attn_activation_cache['hook_k'] = keys
        self.attn_activation_cache['hook_attn_scores'] = attn_scores
        self.attn_activation_cache['hook_pattern'] = softmax_attn
        self.attn_activation_cache['hook_v'] = values
        self.attn_activation_cache['hook_z'] = z
        self.attn_activation_cache['hook_attn_out'] = output

        return output

    def apply_causal_mask(self, attn_scores: t.Tensor):
        # attn_scores: [batch, n_heads, query_pos, key_pos]
        # output: [batch, n_heads, query_pos, key_pos]
        mask = t.triu(attn_scores, diagonal=1).bool()
        return attn_scores.masked_fill_(mask, self.IGNORE)

define a one layer attn only class      

In [9]:
class OneLayerAttnOnly(nn.Module):
    def __init__(self, model_cfg):
        super().__init__()
        self.model_cfg = model_cfg
        self.embed = OneHotEmbed(model_cfg)
        self.attn = Attention(model_cfg)
        self.unembed = OneHotUnembed(model_cfg)

        self.activation_cache = {}


    def forward(self, tokens):
        # tokens [batch, pos]
        # output [batch, pos, d_vocab]

        input = tokens.to(device)
        resid_pre = self.embed(input)
        resid_post = resid_pre + self.attn(resid_pre)
        out = self.unembed(resid_post)

        # add activations to the same cache as before
        self.activation_cache = self.attn.attn_activation_cache
        self.activation_cache['hook_resid_pre'] = resid_pre
        self.activation_cache['hook_resid_post'] = resid_post
        self.activation_cache['unembed'] = out

        return out
    

attn_only_1l = OneLayerAttnOnly(model_cfg).to(device)
attn_only_1l

        

OneLayerAttnOnly(
  (embed): OneHotEmbed(
    (embedding): Embedding(12, 12)
  )
  (attn): Attention()
  (unembed): OneHotUnembed()
)

In [10]:
def plot_metrics(train_loss,train_acc, test_loss, test_acc):
    fig = make_subplots(rows=2,cols=2, subplot_titles = ['Train Loss', 'Train Accuracy', 'Test Loss', 'Test Accuracy'])
    fig.add_trace(go.Scatter(y=train_loss), row=1, col=1)
    fig.add_trace(go.Scatter(y=train_acc), row=1, col=2)
    fig.add_trace(go.Scatter(y=test_loss), row=2, col=1)
    fig.add_trace(go.Scatter(y=test_acc), row=2, col=2)
    # get rid of legend
    fig.update_layout(showlegend=False)
    ## add x and y axis labels and change plot style 
    fig.update_xaxes(
        title = 'Epoch',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='black',
        showgrid=False
    )
    fig.update_yaxes(
        title = 'Loss',
        showgrid=False,
        col=1
    )
    fig.update_yaxes(
        title = 'Accuracy',
        showgrid=False,
        col=2
    )
    fig.update_layout(
        font = dict(size =12),
        width=800,
        height =500,
        margin=dict(l=10, 
                    r = 20,
                    b=10,
                    t=20,
                    pad=0)
    )
    fig.show()

In [11]:
@dataclass
class train_cfg:
    ctx_length: int = 4
    trainset_size: int = 100000
    valset_size: int = 1000
    prob: int = 2
    epochs: int = 15
    batch_size: int = 512
    weight_decay: float = 0.0
    seed: int = 42
    lr: float = 1e-3

class Trainer:

    def __init__(self, train_cfg, model_cfg):
        self.train_cfg = train_cfg
        self.model = OneLayerAttnOnly(model_cfg).to(device)


    def train_dataloader(self, seed: int):
        trainset = SkipTrigramDataset(size=self.train_cfg.trainset_size,
                                      prob=self.train_cfg.prob,
                                      prompt_length=self.train_cfg.ctx_length)

        return DataLoader(trainset, batch_size=self.train_cfg.batch_size, shuffle=True)

    def val_dataloader(self, seed: int):
        valset = SkipTrigramDataset(size=self.train_cfg.valset_size,
                                    prob = self.train_cfg.prob,
                                    prompt_length=self.train_cfg.ctx_length)
        return DataLoader(valset, batch_size=self.train_cfg.batch_size, shuffle=False)

    def configure_optimizers(self):
        optimizer = t.optim.Adam(self.model.parameters(), lr=self.train_cfg.lr, weight_decay=self.train_cfg.weight_decay)
        return optimizer

In [67]:



def train(model, train_data, criterion, optimizer):

    avg_loss = 0
    acc = 0

    for _, batch in list(enumerate(train_data)):

        X = batch[0].to(device)
        y = batch[1].squeeze().to(device)
        assert X.shape[0] == y.shape[0]

        output = model(X)[:,-1]

        with t.no_grad():
            corr = (output.argmax(dim=-1) == y).count_nonzero()
        loss = criterion(output,y)
        loss.backward()

        optimizer.step()
        avg_loss += loss.item() * len(batch)
        acc += corr.item()
        optimizer.zero_grad()

    cache = model.activation_cache

    return avg_loss/train_cfg.trainset_size, acc/train_cfg.trainset_size, cache
    
def test(model, test_data, criterion):

    avg_loss = 0
    acc = 0

    for _, batch in list(enumerate(test_data)):
        X = batch[0].to(device)
        y = batch[1].squeeze().to(device)
        assert X.shape[0] == y.shape[0]

        # don't log the gradients for testing
        with t.no_grad():
            output = model(X)[:,-1]
        loss = criterion(output,y)
        corr = (output.argmax(dim=-1) == y).count_nonzero()
        avg_loss += loss.item() * len(batch)
        acc += corr.item()


    return avg_loss/train_cfg.valset_size, acc/train_cfg.valset_size

In [68]:
from tqdm import tqdm 

def train_and_eval(train_cfg, model_cfg, save_caches = []):

    trainer = Trainer(train_cfg, model_cfg)
    model = trainer.model
    train_data = trainer.train_dataloader(seed=train_cfg.seed)

    test_data = trainer.val_dataloader(seed=train_cfg.seed)
    criterion = nn.CrossEntropyLoss()
    optimizer = trainer.configure_optimizers()
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []

    caches = []

    for n in tqdm(range(train_cfg.epochs)):
        loss, acc, cache = train(model, train_data, criterion, optimizer)

        train_loss.append(loss)
        train_acc.append(acc)
        t_loss, t_acc = test(model, test_data, criterion)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        print(f"train loss = {loss}, train acc = {acc}, test loss = {t_loss}, test acc = {t_acc}")

        if n in save_caches:
            caches.append(cache)

    return train_loss, train_acc, test_loss, test_acc, caches, trainer.model




Helper functions for plotting

In [69]:
from transformer_lens import utils, HookedTransformer, ActivationCache

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [70]:
def get_head_to_resid(model, cache, head, seq):
  model(seq)
  z = cache["hook_z"][:, :, head, :]
  wo = model.attn.W_O[head, :, :]
  bo = model.attn.b_O
  out = einsum("batch pos d_head, d_head d_model -> batch pos d_model", z, wo) + bo
  return out

def get_head_to_logits(model, head_to_resid):
  return model.unembed(head_to_resid)

def show_pattern(cache, seq, model):
    X = t.Tensor(seq).unsqueeze(0).int()
    model(X)
    ax_label = [f"{i}: {z.item()}" for i, z in enumerate(X[0])]
    imshow(cache['hook_pattern'][0], facet_col = 0, x = ax_label, y = ax_label)

def get_head_to_resid(model, cache, head, seq):
  model(seq)
  z = cache["hook_z"][:, :, head, :]
  wo = model.attn.W_O[head, :, :]
  bo = model.attn.b_O
  out = einsum("batch pos d_head, d_head d_model -> batch pos d_model", z, wo) + bo
  return out

def get_head_to_logits(model, head_to_resid):
  return model.unembed(head_to_resid)





Running for many iterations of d_head

In [94]:
train_cfg.lr

0.001

In [81]:
train_losses = {}
train_accs = {}
test_losses = {}
test_accs = {}
models = {}
#for dh in [1,2,3,4,5]:
for dh in range(1,6):

    train_config = train_cfg(
        ctx_length=4,
        weight_decay=0,
        trainset_size = 100000,
        prob = 2,
        epochs = 60,
        batch_size = 512
    )

    model_config = model_cfg(
        d_model = 11,
        d_vocab = 11,
        d_head = dh,
        init_range = 0.02,
        n_ctx = 4,
        n_heads = 2,
        n_layers = 1)

    train_losses[dh], train_accs[dh], test_losses[dh], test_accs[dh], _, models[dh] = train_and_eval(train_config, model_config)

  2%|▏         | 1/60 [00:00<00:25,  2.30it/s]

train loss = 0.009543825511932374, train acc = 0.07678, test loss = 0.009621893882751465, test acc = 0.084


  3%|▎         | 2/60 [00:00<00:25,  2.30it/s]

train loss = 0.009316181807518005, train acc = 0.12629, test loss = 0.009398043632507324, test acc = 0.144


  5%|▌         | 3/60 [00:01<00:24,  2.28it/s]

...

train loss = 0.0047106051921844486, train acc = 0.54573, test loss = 0.004798409700393677, test acc = 0.557


 98%|█████████▊| 59/60 [00:32<00:00,  1.71it/s]

train loss = 0.004712594873905182, train acc = 0.54629, test loss = 0.004800342798233033, test acc = 0.552


100%|██████████| 60/60 [00:33<00:00,  1.80it/s]

train loss = 0.004710559675693512, train acc = 0.54626, test loss = 0.004805691242218017, test acc = 0.538


In [74]:
ans_map

{(1, 1): 0,
 (1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (2, 2): 4,
 (2, 3): 5,
 (2, 4): 6,
 (3, 3): 7,
 (3, 4): 8,
 (4, 4): 9}

In [82]:
def get_rankings(seq, model):
    keys = seq[:-1]
    X = t.Tensor(seq).unsqueeze(0).int()
    logits = model(X)
    scores = model.activation_cache['hook_attn_scores'][0]
    rankings = {}

    for head in range(model_config.n_heads):
        ## get scores of attention from last query pos to all other (key) positions
        rankings[head] = sorted(list(zip(range(len(keys)),keys, scores[head,-1,:-1].detach())), key = lambda x: x[-1], reverse = True)

    return rankings
    

In [85]:
indx = list(ans_map.keys())
indx.append("nan")
x_labels = [str(x) for x in indx]

for dh in range(1,6):
    pre_trained = models[dh]
    head_outputs = t.zeros((model_config.n_heads, model_config.n_ctx, model_config.d_vocab))

    ## getting DLA from each token to each possible continuation
    for i in range(1,5):
        seq = [i,0]
        X = t.Tensor(seq).unsqueeze(0).int()
        for head in range(model_config.n_heads):
            temp = get_head_to_resid(pre_trained, pre_trained.activation_cache, head, X)
            temp = get_head_to_logits(pre_trained, temp)
            #print(head_outputs[head, i-1].shape, temp[0,-1].shape)
            head_outputs[head, i-1,:] = temp[0,-1,:]
    
    seq = [1,2,3,4,0]
    rankings = get_rankings(seq, pre_trained)


    ## getting attention of each token per head


    print(f"Attention ranking for d_head = {dh} is {rankings}")
    imshow(head_outputs, facet_col = 0, y = [1,2,3,4], x = x_labels, title = f"Head Outputs Conditional on Attn for d_head = {dh}",xaxis = "Logit contribution to completions", yaxis = "Source token")

Attention ranking for d_head = 1 is {0: [(3, 4, tensor(10.7204)), (0, 1, tensor(10.5642)), (2, 3, tensor(2.6849)), (1, 2, tensor(2.4218))], 1: [(2, 3, tensor(11.4649)), (0, 1, tensor(3.0419)), (1, 2, tensor(3.0279)), (3, 4, tensor(2.1791))]}


Attention ranking for d_head = 2 is {0: [(0, 1, tensor(10.3109)), (3, 4, tensor(9.4504)), (1, 2, tensor(5.5252)), (2, 3, tensor(-1.1040))], 1: [(2, 3, tensor(11.0803)), (1, 2, tensor(10.9012)), (3, 4, tensor(5.5689)), (0, 1, tensor(-2.3904))]}


Attention ranking for d_head = 3 is {0: [(0, 1, tensor(19.8612)), (1, 2, tensor(14.4581)), (3, 4, tensor(5.2263)), (2, 3, tensor(-2.5995))], 1: [(2, 3, tensor(11.7716)), (3, 4, tensor(9.9533)), (1, 2, tensor(8.4585)), (0, 1, tensor(-1.5127))]}


Attention ranking for d_head = 4 is {0: [(1, 2, tensor(22.8451)), (0, 1, tensor(11.3628)), (2, 3, tensor(4.5868)), (3, 4, tensor(-4.1208))], 1: [(3, 4, tensor(20.1046)), (2, 3, tensor(16.7878)), (0, 1, tensor(7.9085)), (1, 2, tensor(-2.9431))]}


Attention ranking for d_head = 5 is {0: [(1, 2, tensor(20.5462)), (0, 1, tensor(11.0641)), (3, 4, tensor(3.4046)), (2, 3, tensor(-3.5068))], 1: [(2, 3, tensor(22.3207)), (3, 4, tensor(12.2126)), (0, 1, tensor(3.2533)), (1, 2, tensor(-7.4202))]}


In [88]:
## save models where heads share attention to the same source token as a different way of acheiving the task

#t.save(models[1], 'dhead1_new.pth')


In [96]:
ans_map

{(1, 1): 0,
 (1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (2, 2): 4,
 (2, 3): 5,
 (2, 4): 6,
 (3, 3): 7,
 (3, 4): 8,
 (4, 4): 9}

In [164]:
def get_scores(model, seq):
    X = t.Tensor(seq).unsqueeze(0).int()
    logits = model(X)
    scores = model.activation_cache['hook_attn_scores'][0]
    return scores

seq = [1,2,3,4,0]

test_2 = t.load('saved_models/dhead2_new.pth')
test_1 = t.load('saved_models/dhead1_new.pth')


print(t.stack([get_scores(test_1, seq)[head, -1, :-1].detach() for head in range(model_config.n_heads)]))

print(t.stack([get_scores(test_2, seq)[head, -1, :-1].detach() for head in range(model_config.n_heads)]))
#imshow(attn_list, facet_col = 0,  title = f"Attention Scores per Source Token for d_head = {1}")

tensor([[10.5642,  2.4218,  2.6849, 10.7204],
        [ 3.0419,  3.0279, 11.4649,  2.1791]])
tensor([[ -7.8753,  11.1438,  13.2931,  11.7319],
        [ 19.8794,  20.3196, -18.3274,   8.4864]])


In [160]:
seq = [2,5,3,0]


show_pattern(test_2.activation_cache, seq, test_2)
show_pattern(test_1.activation_cache, seq, test_1)

# for i in range(2,6):
#     show_pattern(models[i].activation_cache, seq, models[i])

In [158]:
seq = [1,2,3,4,0]
print(get_rankings(seq, test_1))
get_rankings(seq, test_2)

{0: [(3, 4, tensor(10.7204)), (0, 1, tensor(10.5642)), (2, 3, tensor(2.6849)), (1, 2, tensor(2.4218))], 1: [(2, 3, tensor(11.4649)), (0, 1, tensor(3.0419)), (1, 2, tensor(3.0279)), (3, 4, tensor(2.1791))]}


{0: [(2, 3, tensor(13.2931)),
  (3, 4, tensor(11.7319)),
  (1, 2, tensor(11.1438)),
  (0, 1, tensor(-7.8753))],
 1: [(1, 2, tensor(20.3196)),
  (0, 1, tensor(19.8794)),
  (3, 4, tensor(8.4864)),
  (2, 3, tensor(-18.3274))]}

For binomial distribution (k=1 Bernoulli trials), standard error for a sample stat is \sqrt(pq/n) ~ .015 for the val set, so possibly overfitting is pulling us out of 

In [90]:
test_err = np.sqrt(1/(4*train_config.valset_size))

train_err = np.sqrt(1/(4*train_config.trainset_size))
for ind in range(1,6):
    #print(train_losses[ind][-1],train_accs[ind][-1], test_losses[ind][-1], test_accs[ind][-1])
    print(f"For d_head = {ind}, the final test accuracy is {test_accs[ind][-1]}, the max test accuracy is {np.max(test_accs[ind])} at epoch {np.argmax(test_accs[ind])}/{train_config.epochs}")
    print(f"For d_head = {ind}, the final training accuracy is {train_accs[ind][-1]}, the max test accuracy is {np.max(train_accs[ind])} at epoch {np.argmax(train_accs[ind])}/{train_config.epochs}")

    print(f"test error within range? {np.abs(test_accs[ind][-1] - np.max(test_accs[ind])) <= test_err}")

    print(f"train error within range? {np.abs(train_accs[ind][-1] - np.max(train_accs[ind])) <= test_err}")
    plot_metrics(train_losses[ind],train_accs[ind], test_losses[ind], test_accs[ind]) 

For d_head = 1, the final test accuracy is 0.547, the max test accuracy is 0.547 at epoch 24/60
For d_head = 1, the final training accuracy is 0.54463, the max test accuracy is 0.54476 at epoch 25/60
test error within range? True
train error within range? True


For d_head = 2, the final test accuracy is 0.549, the max test accuracy is 0.551 at epoch 50/60
For d_head = 2, the final training accuracy is 0.54461, the max test accuracy is 0.54499 at epoch 12/60
test error within range? True
train error within range? True


For d_head = 3, the final test accuracy is 0.551, the max test accuracy is 0.559 at epoch 38/60
For d_head = 3, the final training accuracy is 0.54547, the max test accuracy is 0.54613 at epoch 40/60
test error within range? True
train error within range? True


For d_head = 4, the final test accuracy is 0.56, the max test accuracy is 0.562 at epoch 57/60
For d_head = 4, the final training accuracy is 0.54537, the max test accuracy is 0.5457 at epoch 50/60
test error within range? True
train error within range? True


For d_head = 5, the final test accuracy is 0.538, the max test accuracy is 0.557 at epoch 44/60
For d_head = 5, the final training accuracy is 0.54626, the max test accuracy is 0.54682 at epoch 50/60
test error within range? False
train error within range? True


In [91]:
test_signal = SkipTrigramDataset(size = 10000,
                                     prob = 1,
                                     prompt_length = train_config.ctx_length)

for dh in [1,2,3,4,5]:
    model = models[dh]
    acc = 0

    for i in range(len(test_signal)):
        X = test_signal[i][0].unsqueeze(0) # add batch dim to feed into model
        y = test_signal[i][1].item()

        output = model(X)[:,-1]
        corr = (output.argmax(dim=-1) == y).count_nonzero()
        acc += corr.item()

    print(acc/len(test_signal))




1.0
1.0
1.0
1.0
1.0


In [92]:
seq = [3,5,2,0]
for i in range(1,6):
    print(get_rankings(seq, models[i]))
    show_pattern(models[i].activation_cache, seq, models[i])

{0: [(0, 3, tensor(2.6849)), (2, 2, tensor(2.4218)), (1, 5, tensor(-7.2294))], 1: [(0, 3, tensor(11.4649)), (2, 2, tensor(3.0279)), (1, 5, tensor(-3.0561))]}


{0: [(2, 2, tensor(5.5252)), (0, 3, tensor(-1.1040)), (1, 5, tensor(-6.0368))], 1: [(0, 3, tensor(11.0803)), (2, 2, tensor(10.9012)), (1, 5, tensor(-7.2460))]}


{0: [(2, 2, tensor(14.4581)), (0, 3, tensor(-2.5995)), (1, 5, tensor(-10.5149))], 1: [(0, 3, tensor(11.7716)), (2, 2, tensor(8.4585)), (1, 5, tensor(-12.0351))]}


{0: [(2, 2, tensor(22.8451)), (0, 3, tensor(4.5868)), (1, 5, tensor(-14.3393))], 1: [(0, 3, tensor(16.7878)), (2, 2, tensor(-2.9431)), (1, 5, tensor(-20.8075))]}


{0: [(2, 2, tensor(20.5462)), (0, 3, tensor(-3.5068)), (1, 5, tensor(-15.0189))], 1: [(0, 3, tensor(22.3207)), (2, 2, tensor(-7.4202)), (1, 5, tensor(-15.3385))]}


Does the QK matrix store things orthogonally? (It's not sparse?)

In [127]:
def cos_sim(v1, v2):
    with t.no_grad():
        out = einsum("d, d -> ", v1, v2)/(t.norm(v1) * t.norm(v2))
    return out

assert cos_sim(t.tensor([1.,0.,0.]), t.tensor([0.,4.,0.])) == t.tensor(0.)
assert cos_sim(t.tensor([1.,0.,0.]), t.tensor([2.,0.,0.])) == t.tensor(1.)

In [152]:
## Queries have shape [batch, pos_q, n_heads, d_head]
## Keys have shape [batch, pos_k, n_heads, d_head]
model = models[5]

queries = rearrange(model.activation_cache['hook_q'][0], "q nh dh -> nh q dh") #remove batch dim and rearrange so that d_head is first
keys = rearrange(model.activation_cache['hook_k'][0], "q nh dh -> nh q dh") #remove batch dim and rearrange so that d_head is first

## get the cosim for all pairs of keys and queries

head_idx = 0
[[cos_sim(keys[head_idx,i], queries[head_idx, j]) for j in range(queries.shape[1])] for i in range(keys.shape[1])] ## [pos_k, pos_q]


[[tensor(0.9921), tensor(0.8856), tensor(0.9923), tensor(0.9983)],
 [tensor(-0.8149), tensor(-0.4114), tensor(-0.8022), tensor(-0.7912)],
 [tensor(0.9962), tensor(0.8557), tensor(0.9952), tensor(0.9989)],
 [tensor(-0.9923), tensor(-0.8798), tensor(-0.9922), tensor(-0.9984)]]